说明：明文转码成十六进制后的那个4*4矩阵S，按理说s0,s1,s2,s3这四个字节应该是同一列的关系[[s0...],[s1...],[s2...],[s3...]]，

但我为了方便是[[s0,s1,s2,s3],[...],[...],[...]]这样表示的。本算法里的4*4矩阵S实际上是S.T，可是列混淆矩阵的表示方法是按规定来的。

这就导致了我的程序加密后得到的密文可能与答案不一样，但是解密是完全没问题的。


In [2]:
from numpy import *
class AES:
    Rcon=[0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]
    Sbox=array([[ 99, 124, 119, 123, 242, 107, 111, 197,  48,   1, 103,  43, 254,
            215, 171, 118],
           [202, 130, 201, 125, 250,  89,  71, 240, 173, 212, 162, 175, 156,
            164, 114, 192],
           [183, 253, 147,  38,  54,  63, 247, 204,  52, 165, 229, 241, 113,
            216,  49,  21],
           [  4, 199,  35, 195,  24, 150,   5, 154,   7,  18, 128, 226, 235,
             39, 178, 117],
           [  9, 131,  44,  26,  27, 110,  90, 160,  82,  59, 214, 179,  41,
            227,  47, 132],
           [ 83, 209,   0, 237,  32, 252, 177,  91, 106, 203, 190,  57,  74,
             76,  88, 207],
           [208, 239, 170, 251,  67,  77,  51, 133,  69, 249,   2, 127,  80,
             60, 159, 168],
           [ 81, 163,  64, 143, 146, 157,  56, 245, 188, 182, 218,  33,  16,
            255, 243, 210],
           [205,  12,  19, 236,  95, 151,  68,  23, 196, 167, 126,  61, 100,
             93,  25, 115],
           [ 96, 129,  79, 220,  34,  42, 144, 136,  70, 238, 184,  20, 222,
             94,  11, 219],
           [224,  50,  58,  10,  73,   6,  36,  92, 194, 211, 172,  98, 145,
            149, 228, 121],
           [231, 200,  55, 109, 141, 213,  78, 169, 108,  86, 244, 234, 101,
            122, 174,   8],
           [186, 120,  37,  46,  28, 166, 180, 198, 232, 221, 116,  31,  75,
            189, 139, 138],
           [112,  62, 181, 102,  72,   3, 246,  14,  97,  53,  87, 185, 134,
            193,  29, 158],
           [225, 248, 152,  17, 105, 217, 142, 148, 155,  30, 135, 233, 206,
             85,  40, 223],
           [140, 161, 137,  13, 191, 230,  66, 104,  65, 153,  45,  15, 176,
             84, 187,  22]])
    InvSbox=array([[ 82,   9, 106, 213,  48,  54, 165,  56, 191,  64, 163, 158, 129,
        243, 215, 251],
       [124, 227,  57, 130, 155,  47, 255, 135,  52, 142,  67,  68, 196,
        222, 233, 203],
       [ 84, 123, 148,  50, 166, 194,  35,  61, 238,  76, 149,  11,  66,
        250, 195,  78],
       [  8,  46, 161, 102,  40, 217,  36, 178, 118,  91, 162,  73, 109,
        139, 209,  37],
       [114, 248, 246, 100, 134, 104, 152,  22, 212, 164,  92, 204,  93,
        101, 182, 146],
       [108, 112,  72,  80, 253, 237, 185, 218,  94,  21,  70,  87, 167,
        141, 157, 132],
       [144, 216, 171,   0, 140, 188, 211,  10, 247, 228,  88,   5, 184,
        179,  69,   6],
       [208,  44,  30, 143, 202,  63,  15,   2, 193, 175, 189,   3,   1,
         19, 138, 107],
       [ 58, 145,  17,  65,  79, 103, 220, 234, 151, 242, 207, 206, 240,
        180, 230, 115],
       [150, 172, 116,  34, 231, 173,  53, 133, 226, 249,  55, 232,  28,
        117, 223, 110],
       [ 71, 241,  26, 113,  29,  41, 197, 137, 111, 183,  98,  14, 170,
         24, 190,  27],
       [252,  86,  62,  75, 198, 210, 121,  32, 154, 219, 192, 254, 120,
        205,  90, 244],
       [ 31, 221, 168,  51, 136,   7, 199,  49, 177,  18,  16,  89,  39,
        128, 236,  95],
       [ 96,  81, 127, 169,  25, 181,  74,  13,  45, 229, 122, 159, 147,
        201, 156, 239],
       [160, 224,  59,  77, 174,  42, 245, 176, 200, 235, 187,  60, 131,
         83, 153,  97],
       [ 23,  43,   4, 126, 186, 119, 214,  38, 225, 105,  20,  99,  85,
         33,  12, 125]])
    mix_matrix=array([[0x02,0x03,0x01,0x01],
                          [0x01,0x02,0x03,0x01],##########考虑转置
                          [0x01,0x01,0x02,0x03],
                          [0x03,0x01,0x01,0x02]])
    Invmix_mat=array([[0x0E,0x0B,0x0D,0x09],
                          [0x09,0x0E,0x0B,0x0D],
                          [0x0D,0x09,0x0E,0x0B],
                          [0x0B,0x0D,0x09,0x0E]])
    def __init__(self,num=128):
        pass        
    #将 明文/密文/密钥 读入矩阵
    def read(self,mingwen):#mingwen为list类型
        matrix=[]
        for i in range(16):
            byte=ord(mingwen[i])
            if i % 4 == 0:
                matrix.append([byte])
            else:
                matrix[i // 4].append(byte)
        matrix=array(matrix)
        return matrix      #matrix为array类型
    #把数字形式的密文矩阵转码成字母形式的密文方阵
    def output(self,code):#code为由十进制数组成的方阵
        t=zeros(code.shape,dtype=object)#object太关键了！！！！！！！！！！！
        for i in range(4):
            for j in range(4):
                t[j,i]=chr(code[j,i])
        t=t.reshape(1,-1)
        t=list(t[0])
        return t
            
#####密钥扩展#####################################################
    ##g（又称T）函数
    def T(self,w_i,i):
        lun=i
        t=[w_i[1],w_i[2],w_i[3],w_i[0]]#移位
        #查表Sbox
        x=[];y=[]
        for i in range(4):
            x.append(t[i]&0x0f) #x 列值
            y.append((t[i]&0xf0)>>4)#y 行值
        ss=[]
        for i in range(4):
            ss.append(AES.Sbox[y[i],x[i]])
        #第一个字节与Rcon异或
        ss[0]=ss[0]^AES.Rcon[lun]
        return ss
    ##w(i)扩展--（i>4情况）
    def expand(self,w,i,lun):#w得是array
        dw=array([0,0,0,0],dtype=int32)
        if i%4!=0:
            a=array(w[i-4])
            b=array(w[i-1])
            dw=bitwise_xor(a,b)
        else:
            a=array(w[i-4])
            b=array(self.T(w[i-1],lun))
            dw=bitwise_xor(a,b)
        return dw
    ##获得44*4的密钥扩展
    def key_expand(self,K):
        w=[]
        tem=array(K)
        for i in range(4):
            byte=list(tem[i,:])
            w.append(byte)
        for i in range(4,44):
            lun=i//4-1
            w=array(w)
            dw=self.expand(w,i,lun)
            w=append(w,array([dw]),axis=0)
        return w
#####################################################################

##############加密################加密###################加密########             
        
    #轮密钥加
    def Lmyj(self,s,w,i):
        t=[]
        for j in range(4):
            a=array(w[i%4+j])
            b=array(s[j])
            m=list(bitwise_xor(a,b))
            t.append(m)
        return t   #t 为 16字节的0维list
    #字节代换
    def byte_sub(self,s):#s需要是array类型
        t=zeros(s.shape,dtype=int32)
        #查表Sbox
        x=zeros(s.shape,dtype=int32);y=zeros(s.shape,dtype=int32)
        for i in range(4):
            for j in range (4):
                x[j,i]=(s[j,i]&0x0f) #x 列值
                y[j,i]=(s[j,i]&0xf0)>>4#y 行值
                t[j,i]=AES.Sbox[y[j,i],x[j,i]]#查表
        return t      #t是array类型
    #行移位
    def row_shift(self,s):#s   array类型
        s_T=s.T
        t=zeros(s.shape,dtype=int32)
        for j in range(4):
            for i in range(3,-1,-1):
                t[j,i]=s_T[j,int32((i+j)%4)]
        t=t.T
        return t#   array类型
  #################GF(2）域乘法功能的实现################ 

    def find_max(self,byte):#寻找一个数的最高二进制位，为mod函数的子函数
        max=0
        for i in range(16):
            
            t=int32(0x8000/(2**i))
            #print("t=",t ,type(t), byte,type(byte))
            if (byte)&(t):
                max=t
                break
            elif (not(byte&t))&(i==15):
                max=0
            else:
                continue
        return max
    
    def mod(self,byte):#取余，为mul函数的子函数
        m=0x11b
        m_max=0b100000000
        if self.find_max(byte)>=m_max:
            bb=byte
            while self.find_max(bb)>=m_max:
               # print("incycle")
                t=int32(self.find_max(bb)/m_max)
               
                bb=bb^(m*t)
            #print("不是死循环")
            result=bb
        else:
            result=byte
        return result
    
    def mul(self,byte1,byte2):
        sum=0
        for i in range(16):
            #print(i)
            t=byte2&(2**i)
            #print(byte2,type(byte2),type(2**i))
            t=t*byte1
            #print(t)
            sum=sum^t
            #print("sum",sum)
            #print("完成一次乘法")
        sum=self.mod(sum)
        return sum
   #############################################       
    #列混淆
    def s_ji(self,s,j,i):
        mat=AES.mix_matrix
        sum=0
        for ii in range(4):
            sum=sum^self.mul(mat[j,ii],s[ii,i])
        return sum
        
    def col_mix(self,s):
        s_=zeros(s.shape,dtype=int32)
        for i in range(4):
            for j in range(4):
                s_[j,i]=self.s_ji(s,j,i)
        return s_
#####################################################################

##############解密################解密###################解密########
    #逆字节代换
    def Invbyte_sub(self,s):#s需要是array类型
        t=zeros(s.shape,dtype=int32)
        #查表InvSbox
        x=zeros(s.shape,dtype=int32);y=zeros(s.shape,dtype=int32)
        for i in range(4):
            for j in range (4):
                x[j,i]=(s[j,i]&0x0f) #x 列值
                y[j,i]=(s[j,i]&0xf0)>>4#y 行值
                t[j,i]=AES.InvSbox[y[j,i],x[j,i]]#查表
        return t      #t是array类型
    
    #逆行移位
    def Invrow_shift(self,s):#s   array类型
        s_T=s.T
        t=zeros(s.shape,dtype=int32)
        for j in range(4):
            for i in range(4):
                t[j,i]=s_T[j,int32((i-j)%4)]
        t=t.T
        return t#   array类型
    
    #逆列混合 
    def Invs_ji(self,s,j,i):#j代表行  i代表列
        mat=AES.Invmix_mat
        sum=0
        for ii in range(4):
            sum=sum^self.mul(mat[j,ii],s[ii,i])
        return sum
        
    def Invcol_mix(self,s):
        s_=zeros(s.shape,dtype=int32)
        for i in range(4):
            for j in range(4):
                s_[j,i]=self.Invs_ji(s,j,i)
        return s_
    
    
        

T1.本题使用字符串'abcdefghijklmn'作为输入，完成字符串的补位、转码、字节替代、逆字节
替代、转码、去补位，输出每一步的结果。


In [3]:
s='abcdefghijklmn'
######补位
l=len(s)
if l%16!=0:
    num_zero=16-l%16
else:
    num_zero=0
ss=[]
for i in range(l):
    ss.append(s[i])
for j in range(num_zero):
    ss.append("0")
print("补位：\n",ss)
######转码
aes=AES()
sss=aes.read(ss)

print("转码:\n",sss)
######字节替代
s4=aes.byte_sub(sss)
print("字节替代:\n",s4.T)
######逆字节替代      
s5=aes.Invbyte_sub(s4)
print("逆字节替代:\n",s5.T)
######转码
s6=aes.output(s5)
print("转码输出:\n",s6)
######去补位
#s7=array(s6)
s7=s6[0:-num_zero]
s7=list(s7)
#string=""
#s7=string.join(s7)
print("去补位:\n",s7)

补位：
 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', '0', '0']
转码:
 [[ 97  98  99 100]
 [101 102 103 104]
 [105 106 107 108]
 [109 110  48  48]]
字节替代:
 [[239  77 249  60]
 [170  51   2 159]
 [251 133 127   4]
 [ 67  69  80   4]]
逆字节替代:
 [[ 97 101 105 109]
 [ 98 102 106 110]
 [ 99 103 107  48]
 [100 104 108  48]]
转码输出:
 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', '0', '0']
去补位:
 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n']


T2.行移位变换和逆行移位变换，列混合和逆列混合
本题使用字符串'abcdefghijklmnop'作为验证输入，将其转换为字节矩阵后对四个变换进行
验证，记录其输出。


In [4]:

s='abcdefghijklmnop'
######转码
aes=AES()
s1=aes.read(s)
print("转码:\n",s1.T)
######行移位变换
s2=aes.row_shift(s1)
print("行移位变换:\n",s2.T)
######逆行移位变换
s3=aes.Invrow_shift(s2)
print("逆行移位变换:\n",s3.T)
######列混合
st=array(s1,dtype=int32)
s4=aes.col_mix(st)
print("列混合:\n",s4.T)
######逆列混合
s5=aes.Invcol_mix(s4)
print("逆列混合:\n",s5.T)

转码:
 [[ 97 101 105 109]
 [ 98 102 106 110]
 [ 99 103 107 111]
 [100 104 108 112]]
行移位变换:
 [[ 97 101 105 109]
 [102 106 110  98]
 [107 111  99 103]
 [112 100 104 108]]
逆行移位变换:
 [[ 97 101 105 109]
 [ 98 102 106 110]
 [ 99 103 107 111]
 [100 104 108 112]]
列混合:
 [[105 125  97 117]
 [106 126  98 118]
 [107 127  99 119]
 [108 112  68  72]]
逆列混合:
 [[ 97 101 105 109]
 [ 98 102 106 110]
 [ 99 103 107 111]
 [100 104 108 112]]


3.轮密钥生成
使用原始密钥'abcdefghijklmnop'生成总计11组扩展密钥𝑘0~𝑘10，将用于之后的加解密。初
始密钥K转换为字节矩阵后的4列为𝑊0~𝑊3，后续的𝑊4~𝑊43使用递归计算得到。

In [5]:
s='abcdefghijklmnop'
######转码
aes=AES()
K=aes.read(s)
print("转码:\n",K.T)
######密钥扩展
K_44=aes.key_expand(K)
print("密钥扩展:\n",K_44.T)


转码:
 [[ 97 101 105 109]
 [ 98 102 106 110]
 [ 99 103 107 111]
 [100 104 108 112]]
密钥扩展:
 [[ 97 101 105 109 255 154 243 158  63 165  86 200  14 171 253  53  49 154
  103  82  64 218 189 239   1 219 102 137 225  58  92 213 114  72  20 193
  102  46  58 251  70 104  82 169]
 [ 98 102 106 110 202 172 198 168  27 183 113 217 173  26 107 178  27   1
  106 216 107 106   0 216 245 159 159  71 197  90 197 130 230 188 121 251
  193 125   4 255 174 211 215  40]
 [ 99 103 107 111  50  85  62  81  67  22  40 121  62  40   0 121 129 169
  169 208  15 166  15 223 126 216 215   8  53 237  58  50 216  53  15  61
    1  52  59   6  33  21  46  40]
 [100 104 108 112  88  48  92  44  83  99  63  19 187 216 231 244  45 245
   18 230  45 216 202  44 242  42 224 204  85 127 159  83  86  41 182 229
   46   7 177  84  33  38 151 195]]


4、 加密机和解密机

整合轮函数，完成AES加密和解密，原始密钥使用'abcdefghijklmnop'
明文为 'Cryptography and Network Security;学号;姓名拼音' ，使用自己的学号和姓名拼
音替代对应位置字符串

In [7]:
def encrypt(mingwen,Key):
    #把输入明文字符串每16字符一段存入inputs列表中
    inputs=[]  
    l=len(mingwen)
    l=l//16
    for i in range(l):
        inputs.append(mingwen[(i*16):((i+1)*16)])
    if len(mingwen)%16!=0:
        inputs.append(mingwen[(l*16):])
    output=[]
    one=(len(mingwen)%16)!=0
    #inputs内每一字符串依次加密
    for j in range(l+one):
        ##########预处理############
        s=inputs[j]
        ######补位
        l=len(s)
        if l%16!=0:
            num_zero=16-l%16
        else:
            num_zero=0
        ss=[]
        
        for i in range(l):
            ss.append(s[i])
       
        for j in range(num_zero):
            ss.append("0")
        ######转码
        aes=AES()
        ming_hex=aes.read(ss)
      
        ######密钥扩展
        ####转码
        K=aes.read(Key)
        K_44=aes.key_expand(K)
        #############################
        #轮密钥加 i=0
        st=ming_hex    
        st=aes.Lmyj(st,K_44,0)
        ##########第1-9轮加密########
        for i in range(1,10):
            st=array(st)
            #####字节替代
            st=aes.byte_sub(st)
            ######行移位变换
            st=aes.row_shift(st)
            ######列混合
            st=array(st,dtype=int32)
            st=aes.col_mix(st)
            ######轮密钥加
            st=aes.Lmyj(st,K_44,i)
        ######第10轮加密##########
        #####字节替代
        st=array(st)
        st=aes.byte_sub(st)
        ######行移位变换
        st=aes.row_shift(st)
        ######轮密钥加
        st=aes.Lmyj(st,K_44,10)
        st=array(st)
        ######转码
        miwen_=aes.output(st)
        miwen=miwen_

        output=output+miwen
    out_str="".join(output)
    return (out_str,num_zero)#返回一个元祖，out_str为密文，num_zero为不足16位的字符串补零个数
    

In [8]:
def decrypt(miwen,Key,num_zero):
    #把输入密文字符串每16字符一段存入inputs列表中
    inputs=[]
    l=len(miwen)
    l=l//16
    for i in range(l):
        inputs.append(miwen[(i*16):((i+1)*16)])
    if len(miwen)%16!=0:
        inputs.append(miwen[(l*16):])
    output=[]
    for j in range(l):
        ##########预处理#############
        s=inputs[j]
        ######
        l=len(s)
        ss=[]
        for i in range(l):
            ss.append(s[i])
        ######转码
        aes=AES()
        mi_hex=aes.read(ss)
        ######密钥扩展
        ####转码
        K=aes.read(Key)
        ######密钥扩展
        K_44=aes.key_expand(K)
        #############################
        ############第10轮解密
        st=mi_hex
        ######轮密钥加
        st=aes.Lmyj(st,K_44,10)
        st=array(st)
        ######行移位变换
        st=aes.Invrow_shift(st)
        #####字节替代
        st=aes.Invbyte_sub(st)
        ##########第9-1轮解密########   
        for i in range(9,0,-1):
            st=array(st)
            ######轮密钥加
            st=aes.Lmyj(st,K_44,i)
            st=array(st)
            ######逆列混合
            st=array(st,dtype=int32)
            st=aes.Invcol_mix(st)
            ######逆行移位变换
            st=aes.Invrow_shift(st)
            #####逆字节替代
            st=aes.Invbyte_sub(st)
            
        ##轮密钥加 i=0    
        st=aes.Lmyj(st,K_44,0)
        st=array(st)           
        ######转码
        miwen_=aes.output(st)
        output=output+miwen_
        ######去补位
    if num_zero!=0:
            output=output[0:-num_zero]
    
    out_str="".join(output)
    return out_str

In [13]:
##########AES加密实例
key='abcdefghijklmnop'
mingwen='Cryptography and Network Security;2022214366;MaXiaohui'
res=encrypt(mingwen,key)
a=res[0];num=res[1]
a


"Ë$2\x06ü}\x96¤!Jï6zÁgø½s¼\x80\x19Êº{Û\x91 ü\x01\xa0âWÂ\x8aæ9S»YÛèõ\x99\x19\x8e\x8dáÇ6|=iª'{adkøf[O^!"

In [15]:
##########AES解密实例      #只要明文长度不是16整数倍，就解密错误
key='abcdefghijklmnop'
miwen=a
b=decrypt(miwen,key,num)
b

'Cryptography and Network Security;2022214366;MaXiaohui'

In [16]:
#超长明文加密解密！！！
mingwen='New security threats are generated with the coming of information industrial 5.0 represented by artificial\
intelligence and Industrial Internet of Things (IIoT). One of the most significant concerns in IIoT is information\
security which refers to the protection of the privacy of image owner. To this end, a new reversible data hiding\
in encrypted image (RDHEI) method with high capacity for IIoT security is proposed in this paper. On image\
owner side, image contents are protected by a block level image encryption method while the correlations of\
adjacent pixels inside the same block are not damaged. On receiver side, to realize reversibility, all encrypted\
pixel blocks are classified into overflow blocks and non-overflow blocks and only non-overflow blocks are used\
to carry additional data. Then, some secret data, which is used to manage encrypted images, is embedded\
into the largest and smallest pixels in non-overflow blocks by utilizing pixel-value-ordering and histogram\
shifting techniques. On the receiver side, original image and secret data can be recovered perfectly. Moreover,\
according the availability of decryption key and data hiding key, image recovery and data extraction are\
separably executed. Experimental results demonstrate that significant improvement in embedding capacity is\
achieved without destroying reversibility. In this way, the proposed method outperforms several state-of-the-art\
methods and it is quite suitable for IIoT.' 

key='abcdefghijklmnop'
res=encrypt(mingwen,key)
a=res[0];num=res[1]
a

'c\x96,!\x8ePü\x87\x8aê\x0f\x93nØn\x8b\x9820\x15\x80µz\x01yHË\x81Ci\x17\x14uf³+´\x07J¯R£\x19â\x97\xa0B\x81ë[6£ð%/\xa0»6?s\x9b6f\x17\x90XàÆl\x8e\x8a½\x83æ(udh\x9eIT;4¿Åí\x1fÏ\x08\xa0\x89Ü)qE®\x84þ\x08îJíÌ\xa04tWÊÜ\x1bÚBO\x86È@½9\x8a©\x1c¹b¬\x99!\x81\x9e\x80|\x81\x93\x1eîMö\x92\xa0\x9bó@Íé÷5Ñb¬Æ»[¯¿¸\x105úö\x1c\x14Ø¨ì\x13\x94\x9c±°`Çñ\x95\x81ÑO°\x16»û\x10Â¢\x12`(Ü#ã¹t©È\n5Ö\r\x99\x9f\x8cðìjÃÀ¢pþ\x1f¬4\x1cs®vÒ5lB\n\x93múÈ\x019×¸X\x00ÖÏfO\x0c\x94ÝªÉ\x84ry\x84B¤{ådqþ:ð;3\x19º`@">M\x1bn\x83QfIÉ£Ì¸YÒ×ë\x08\x84§7\x99\x9fòH\x91\x11\x7fëíÑ\x8eñ»\x98OÄxöÛ\x07ÛÑ\x01þpN\\Âíî\x0cÀÿ±D\x8dñ\x01\x12\x99\x80\'\x87é¥zXµ¦ù\x92¹±6ð\x19\x1d\x1c4F¶ã\x96È\x05}²Ôüäú>\x12k\x80µ\x90Ü|è\x887|ð1ö\x95KêÍ\x83§¬m0Z\x9cÇñL\x1f¿éh7\\lü\x0fr¨J\x94Ó\x8aâ&\x94ÅP7\x82\xa0Ù\x990\x08(tÚr\x86\x9asã\x03\xa08:\x91v\x9d\x86óº*>+ÀQ\x9d\x9cëC¤éq®AÏoåþ\x96\xadàýwÎcV\x11Û0Å\x1a\x85\x18\x93Î«a\\\x14u\x05\x85³E±x;\rË¹\x8c\x1dGÞ¼\x93\x14lñ÷í\x05[ðÒs\x87FùûLtâ£\xa0ºoáze\x9d å?ÑØÙ^L|ìP\x11õZýÜG]\x18m\x17Ë\x9ch~\x9d\x01ge\'nó\x16¼â÷ÎO\x87

In [17]:
key='abcdefghijklmnop'
miwen=a
b=decrypt(miwen,key,num)
b

'New security threats are generated with the coming of information industrial 5.0 represented by artificialintelligence and Industrial Internet of Things (IIoT). One of the most significant concerns in IIoT is informationsecurity which refers to the protection of the privacy of image owner. To this end, a new reversible data hidingin encrypted image (RDHEI) method with high capacity for IIoT security is proposed in this paper. On imageowner side, image contents are protected by a block level image encryption method while the correlations ofadjacent pixels inside the same block are not damaged. On receiver side, to realize reversibility, all encryptedpixel blocks are classified into overflow blocks and non-overflow blocks and only non-overflow blocks are usedto carry additional data. Then, some secret data, which is used to manage encrypted images, is embeddedinto the largest and smallest pixels in non-overflow blocks by utilizing pixel-value-ordering and histogramshifting techniques. O